##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treinamento personalizado: tutorial

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/customization/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/customization/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/customization/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Este tutorial mostra como treinar um modelo de aprendizado de máquina com um loop de treinamento personalizado para *categorizar* pinguins por espécie. Neste notebook, você usará o TensorFlow para realizar o seguinte:

1. Importar um dataset
2. Criar um modelo linear simples
3. Treinar o modelo
4. Avaliar a eficácia do modelo
5. Usar o modelo treinado para fazer previsões

## Programação do TensorFlow

Este tutorial demonstra as seguintes tarefas de programação do TensorFlow:

- Importar dados com a [API TensorFlow Datasets](https://www.tensorflow.org/datasets/overview#load_a_dataset)
- Criar modelos e camadas com a [API Keras](https://www.tensorflow.org/guide/keras/)


## Problema de classificação de pinguins

Imagine que você é um ornitólogo em busca de uma maneira automatizada de categorizar todos os pinguins que encontra. O aprendizado de máquina oferece vários algoritmos para classificar pinguins estatisticamente. Por exemplo, um programa de aprendizado de máquina sofisticado pode classificar pinguins com base em fotografias. O modelo que você criará neste tutorial é um pouco mais simples. Ele classifica pinguins com base no peso, no comprimento das nadadeiras e nos bicos, especialmente nas medidas de comprimento e largura do [cúlmen](https://en.wikipedia.org/wiki/Beak#Culmen).

Há 18 espécies de pinguins, mas, neste tutorial, você só tentará classificar estas três:

- Pinguins-de-barbicha
- Pinguins-gentoo
- Pinguins-de-adélia

<table>
  <tr><td>     <img src="https://www.tensorflow.org/tutorials/customization/images/penguins_ds_species.png" class="no-filter" alt="Ilustração de pinguins-de-barbicha, pinguins-gentoo e pinguins-de-adélia"> </td></tr>
  <tr><td align="center">     <b>Figura 1.</b> <a href="https://en.wikipedia.org/wiki/Chinstrap_penguin">Pinguins-de-barbicha</a>, <a href="https://en.wikipedia.org/wiki/Gentoo_penguin">pinguins-gentoo</a> e <a href="https://en.wikipedia.org/wiki/Ad%C3%A9lie_penguin">pinguins-de-adélia</a> (obra de arte de @allison_horst, CC BY-SA 2.0).<br> </td></tr>
</table>

Felizmente, uma equipe de pesquisa já criou e compartilhou um [dataset de 334 pinguins](https://allisonhorst.github.io/palmerpenguins/), com peso, comprimento das nadadeiras, medidas dos bicos e outros dados. Esse dataset também está convenientemente disponível como o TensorFlow Dataset [penguins](https://www.tensorflow.org/datasets/catalog/penguins). 

## Configuração

Instale o pacote `tfds-nightly` para o dataset dos pinguins. O pacote `tfds-nightly` é a versão noturna do TensorFlow Datasets (TFDS). Para mais informações sobre o TFDS, confira a [Visão geral do TensorFlow Datasets](https://www.tensorflow.org/datasets/overview).

In [ ]:
!pip install -q tfds-nightly

Depois, selecione **Runtime &gt; Restart Runtime** no menu do Colab para reiniciar o runtime.

Não prossiga com o resto deste tutorial sem primeiro reiniciar o runtime.

Importe o TensorFlow e os outros módulos necessários do Python. 

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

print("TensorFlow version: {}".format(tf.__version__))
print("TensorFlow Datasets version: ",tfds.__version__)

## Importe o dataset

O TensorFlow Dataset [penguins/processed](https://www.tensorflow.org/datasets/catalog/penguins) padrão já está limpo, normalizado e pronto para a criação de um modelo. Antes de baixar os dados processados, visualize uma versão simplificada para se familiarizar com os dados da pesquisa original sobre os pinguins.


### Visualize os dados

Baixe a versão simplificada do dataset dos pinguins (`penguins/simple`) usando o método [`tdfs.load`](https://www.tensorflow.org/datasets/api_docs/python/tfds/load) do TensorFlow Datasets. Há 344 registros de dados nesse dataset. Extraia os cinco primeiros registros em um objeto [`DataFrame`](https://www.tensorflow.org/datasets/api_docs/python/tfds/as_dataframe) para inspecionar uma amostra dos valores no dataset:

In [ ]:
ds_preview, info = tfds.load('penguins/simple', split='train', with_info=True)
df = tfds.as_dataframe(ds_preview.take(5), info)
print(df)
print(info.features)

As linhas numeradas são os registros de dados, um *[exemplo](https://developers.google.com/machine-learning/glossary/#example)* por linha, em que:

- Os primeiros seis campos são *[características](https://developers.google.com/machine-learning/glossary/#feature)*: eles são as características de um exemplo. Aqui, os campos têm números que representam as medidas dos pinguins.
- A última coluna é o *[rótulo](https://developers.google.com/machine-learning/glossary/#label)*: é o valor que você quer prever. Para esse dataset, é o valor inteiro 0, 1 ou 2, que corresponde ao nome de uma espécie de pinguim.

No dataset, o rótulo para a espécie de pinguim é representado como um número para facilitar o trabalho com o modelo que você está criando. Esses números correspondem às seguintes espécies de pinguins:

- `0`: pinguim-de-adélia
- `1`: pinguim-de-barbicha
- `2`: pinguim-gentoo

Crie uma lista com os nomes das espécies de pinguins nessa ordem. Você usará essa lista para interpretar a saída do modelo de classificação:

In [ ]:
class_names = ['Adélie', 'Chinstrap', 'Gentoo']

Para mais informações sobre características e rótulos, confira a [seção de Terminologia de ML do Curso Intensivo de Aprendizado de Máquina](https://developers.google.com/machine-learning/crash-course/framing/ml-terminology).

### Baixe o dataset pré-processado

Agora, baixe o dataset de pinguins pré-processado (`penguins/processed`) com o método `tfds.load`, que retorna uma lista de objetos `tf.data.Dataset`. O dataset `penguins/processed` não tem o próprio dataset de teste, então use a divisão 80:20 para [separar o dataset completo](https://www.tensorflow.org/datasets/splits) em datasets de treinamento e teste. Depois, você usará o dataset de teste para verificar seu modelo.

In [ ]:
ds_split, info = tfds.load("penguins/processed", split=['train[:20%]', 'train[20%:]'], as_supervised=True, with_info=True)

ds_test = ds_split[0]
ds_train = ds_split[1]
assert isinstance(ds_test, tf.data.Dataset)

print(info.features)
df_test = tfds.as_dataframe(ds_test.take(5), info)
print("Test dataset sample: ")
print(df_test)

df_train = tfds.as_dataframe(ds_train.take(5), info)
print("Train dataset sample: ")
print(df_train)

ds_train_batch = ds_train.batch(32)

Observe que essa versão do dataset foi processada ao reduzir os dados a quatro características normalizadas e um rótulo de espécie. Nesse formato, os dados podem ser usados rapidamente para treinar um modelo sem processamento adicional.

In [ ]:
features, labels = next(iter(ds_train_batch))

print(features)
print(labels)

Visualize alguns grupos ao fazer a plotagem de algumas características do lote:

In [ ]:
plt.scatter(features[:,0],
            features[:,2],
            c=labels,
            cmap='viridis')

plt.xlabel("Body Mass")
plt.ylabel("Culmen Length")
plt.show()

## Crie um modelo linear simples

### Por que um modelo?

Um *[modelo](https://developers.google.com/machine-learning/crash-course/glossary#model)* é uma relação entre características e o rótulo. Para o problema de classificação de pinguins, o modelo define a relação entre a massa corporal, as medidas da nadadeira e do cúlmen e a espécie de pinguim prevista. Alguns modelos simples podem ser descritos com algumas linhas de álgebra, mas modelos de aprendizado de máquina complexos têm um número maior de parâmetros que são difíceis de resumir.

Você consegue determinar a relação entre as quatro características e a espécie de pinguim *sem* usar aprendizado de máquina? Ou seja, você consegue usar técnicas de programação tradicional (por exemplo, várias declarações condicionais) para criar um modelo? Talvez — se você analisar o dataset por tempo suficiente para determinar as relações entre a massa corporal e as medidas do cúlmen de uma determinada espécie. E isso se torna difícil — talvez impossível — em datasets mais complicados. Uma boa abordagem de aprendizado de máquina *determina o modelo para você*. Se você alimentar o tipo de modelo de aprendizado de máquina correto com um número suficiente de exemplos representativos, o programa descobre as relações para você.

### Selecione o modelo

Em seguida, você precisa selecionar o tipo de modelo para treinar. Há vários tipos de modelos e, para escolher um bom, é preciso experiência. Este tutorial usa uma rede neural para resolver o problema de classificação de pinguins. As [*redes neurais*](https://developers.google.com/machine-learning/glossary/#neural_network) conseguem encontrar relações complexas entre características e o rótulo. É um grafo altamente estruturado, organizado em uma ou mais [*camadas ocultas*](https://developers.google.com/machine-learning/glossary/#hidden_layer). Cada camada oculta consiste em um ou mais [*neurônios*](https://developers.google.com/machine-learning/glossary/#neuron). Há várias categorias de redes neurais, e este programa usa uma [*rede neural totalmente conectada*](https://developers.google.com/machine-learning/glossary/#fully_connected_layer) ou densa: os neurônios em uma camada recebem conexões de entrada de *todos* os neurônios na camada anterior. Por exemplo, a Figura 2 ilustra uma rede neural densa que consiste em uma camada de entrada, duas camadas ocultas e uma camada de saída:

<table>
  <tr><td>     <img src="https://www.tensorflow.org/tutorials/customization/images/full_network_penguin.png" class="no-filter" alt="Um diagrama da arquitetura da rede: entradas, 2 camadas ocultas e saídas"> </td></tr>
  <tr><td align="center">     <b>Figura 2.</b> Uma rede neural com características, camadas ocultas e previsões.<br> </td></tr>
</table>

Ao treinar o modelo da Figura 2 e o alimentar com um exemplo não rotulado, são geradas três previsões: a probabilidade que esse pinguim é da espécie específica. Essa previsão é chamada de [*inferência*](https://developers.google.com/machine-learning/crash-course/glossary#inference). Para esse exemplo, a soma das previsões de saída é 1.0. Na Figura 2, essa previsão é detalhada como: `0.02` para *pinguins-de-adélia*, `0.95` para *pinguins-de-barbicha* e `0.03` para *pinguins-gentoo*. Isso significa que o modelo prevê — com 95% de probabilidade — que um exemplo de pinguim não rotulado é um *pinguim-de-barbicha*.

### Crie um modelo usando Keras

A API `tf.keras` do TensorFlow é a maneira recomendada de criar modelos e camadas. Assim, é fácil criar modelos e realizar testes enquanto o Keras lida com a complexidade de conectar tudo.

O modelo `tf.keras.Sequential` é uma pilha linear de camadas. O construtor dele considera uma lista de instâncias de camadas. Nesse caso, duas camadas `tf.keras.layers.Dense`, cada uma com 10 nós, e uma camada de saída com 3 nós, representando suas previsões de rótulos. O parâmetro `input_shape` da primeira camada corresponde ao número de características do dataset e é obrigatório:

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

A [*função activation*](https://developers.google.com/machine-learning/crash-course/glossary#activation_function) determina o formato de saída de cada nó na camada. Essas não linearidades são importantes — sem elas, o modelo seria equivalente a uma única camada. Há várias `tf.keras.activations`, mas a [ReLU](https://developers.google.com/machine-learning/crash-course/glossary#ReLU) é comum para camadas ocultas.

O número ideal de camadas ocultas e neurônios depende do problema e do dataset. Como vários aspectos do aprendizado de máquina, escolher o melhor formato de rede neural exige uma mistura de conhecimento e testes. Como regra, o aumento do número de camadas ocultas e neurônios geralmente cria um modelo mais poderoso, que exige mais dados para treinar de maneira eficaz.

### Use o modelo

Vamos conferir o que esse modelo faz com um lote de características:

In [ ]:
predictions = model(features)
predictions[:5]

Aqui, cada exemplo retorna um [logit](https://developers.google.com/machine-learning/crash-course/glossary#logits) para cada classe.

Para converter esses logits em uma probabilidade para cada classe, use a função [softmax](https://developers.google.com/machine-learning/crash-course/glossary#softmax):

In [ ]:
tf.nn.softmax(predictions[:5])

Ao obter o `tf.math.argmax` das classes, temos o índice de classe previsto. No entanto, o modelo ainda não foi treinado, então as previsões não são boas:

In [ ]:
print("Prediction: {}".format(tf.math.argmax(predictions, axis=1)))
print("    Labels: {}".format(labels))

## Treine o modelo

O [*treinamento*](https://developers.google.com/machine-learning/crash-course/glossary#training) é o estágio do aprendizado de máquina em que o modelo é otimizado gradualmente ou *aprende* o dataset. A meta é aprender o suficiente sobre a estrutura do dataset de treinamento para fazer previsões sobre dados desconhecidos. Se você aprender *demais* sobre o dataset de treinamento, então as previsões só funcionarão para os dados conhecidos e não serão generalizáveis. Esse problema é chamado de [*overfitting*](https://developers.google.com/machine-learning/crash-course/glossary#overfitting), é como memorizar as respostas em vez de entender como resolver um problema.

O problema de classificação de pinguins é um exemplo de [*aprendizado de máquina supervisionado*](https://developers.google.com/machine-learning/glossary/#supervised_machine_learning): o modelo é treinado a partir de exemplos que contêm rótulos. No [*aprendizado de máquina não supervisionado*](https://developers.google.com/machine-learning/glossary/#unsupervised_machine_learning), os exemplos não contêm rótulos. Em vez disso, o modelo geralmente encontra padrões entre as características.

### Defina as funções de perda e gradiente

Em ambos os estágios de treinamento e avaliação, é preciso calcular a [*perda*](https://developers.google.com/machine-learning/crash-course/glossary#loss) do modelo. Isso mede quão distantes as previsões de um modelo estão do rótulo desejado ou, em outras palavras, quão ruim é o desempenho do modelo. Você deve minimizar ou otimizar esse valor.

Seu modelo calculará a perda usando a função `tf.keras.losses.SparseCategoricalCrossentropy`, que pega as previsões de probabilidade de classe do modelo e o rótulo desejado e retorna a perda média dos exemplos.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)

l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Use o contexto `tf.GradientTape` para calcular os [*gradientes*](https://developers.google.com/machine-learning/crash-course/glossary#gradient) usados para otimizar seu modelo:

In [ ]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

### Crie um otimizador

Um [*otimizador*](https://developers.google.com/machine-learning/crash-course/glossary#optimizer) aplica os gradientes computados aos parâmetros do modelo para minimizar a função `loss`. Pense na função de perda como uma superfície curvada (consulte a Figura 3), e você quer encontrar o ponto mais baixo caminhando por ela. Os gradientes apontam na direção da subida mais íngreme, então você percorrerá o caminho oposto e descerá o morro. Ao calcular iterativamente a perda e o gradiente de cada lote, você ajustará o modelo durante o treinamento. Gradualmente, o modelo encontrará a melhor combinação de pesos e bias para minimizar a perda. Quanto menor for a perda, melhores serão as previsões do modelo.

<table>
  <tr><td>     <img src="https://cs231n.github.io/assets/nn3/opt1.gif" width="70%" alt="Algoritmos de otimização visualizados ao longo do tempo em um espaço 3D."> </td></tr>
  <tr><td align="center">     <b>Figura 3.</b> Algoritmos de otimização visualizados ao longo do tempo em um espaço 3D.<br>(Fonte: <a href="http://cs231n.github.io/neural-networks-3/">Stanford class CS231n</a>, Licença MIT, Crédito da imagem: <a href="https://twitter.com/alecrad">Alec Radford</a>)</td></tr>
</table>

O TensorFlow tem vários algoritmos de otimização disponíveis para treinamento, Neste tutorial, você usará o `tf.keras.optimizers.SGD`, que implementa o algoritmo de [*método do gradiente estocástico*](https://developers.google.com/machine-learning/crash-course/glossary#gradient_descent) (SGD). O parâmetro `learning_rate` define o tamanho do passo tomado para cada iteração de descida. Essa taxa é um [*hiperparâmetro*](https://developers.google.com/machine-learning/glossary/#hyperparameter) que você ajustará com frequência para alcançar melhores resultados.

Instancie o otimizador com uma [*taxa de aprendizado*](https://developers.google.com/machine-learning/glossary#learning-rate) de `0.01`, um valor escalar que é multiplicado pelo gradiente em cada iteração do treinamento:

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

Depois, use esse objeto para calcular um único passo de otimização:

In [ ]:
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))

### Loop de treinamento

Com tudo em seu devido lugar, o modelo está pronto para o treinamento! Um loop de treinamento alimenta o modelo com exemplos do dataset para ajudá-lo a fazer previsões. O bloco de código a seguir configura estas etapas de treinamento:

1. Itere cada *época*. Uma época é uma passagem pelo dataset.
2. Em uma época, itere cada exemplo do `Dataset` de treinamento ao pegar as *características* (`x`) e o *rótulo* (`y`).
3. Usando as características do exemplo, faça uma previsão e a compare com o rótulo. Meça a inexatidão da previsão e use isso para calcular a perda e os gradientes do modelo.
4. Use um `optimizer` para atualizar os parâmetros do modelo.
5. Acompanhe algumas estatísticas para visualização.
6. Repita para cada época.

A variável `num_epochs` é o número de loops na coleção do dataset. No código abaixo, `num_epochs` está definido como 201, ou seja, esse loop de treinamento será executado 201 vezes. De maneira contraintuitiva, um treinamento mais longo não garante um modelo melhor. `num_epochs` é um [*hiperparâmetro*](https://developers.google.com/machine-learning/glossary/#hyperparameter) que você pode ajustar. A escolha do número correto geralmente requer experiência e testes.

In [ ]:
## Note: Rerunning this cell uses the same model parameters

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in ds_train_batch:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Como opção, você pode usar o método [`Model.fit(ds_train_batch)`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) built-in do Keras para treinar seu modelo. 

### Visualize a função de perda ao longo do tempo

Embora seja útil imprimir o progresso de treinamento do modelo, você pode visualizá-lo com o [TensorBoard](https://www.tensorflow.org/tensorboard), uma ferramenta de visualização e métricas que acompanha o TensorFlow. Para esse exemplo simples, você criará gráficos básicos usando o módulo `matplotlib`.

A interpretação desses gráficos exige um pouco de experiência, mas, em geral, a *perda* deve cair e a *exatidão* deve aumentar:

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

## Avalie a eficácia do modelo

Agora que o modelo foi treinado, você pode obter algumas estatísticas sobre o desempenho dele.

*Avaliar* significa determinar a eficácia do modelo em fazer previsões. Para determinar a eficácia do modelo na classificação de pinguins, forneça algumas medidas ao modelo e peça para prever quais espécies de pinguins elas representam. Em seguida, compare as previsões do modelo com o rótulo real. Por exemplo, um modelo que escolheu a espécie correta na metade dos exemplos de entrada tem uma [*exatidão*](https://developers.google.com/machine-learning/glossary/#accuracy) de `0.5`. A Figura 4 mostra um modelo um pouco mais eficaz, que acerta 4 a cada 5 previsões, com 80% de exatidão:

<table cellpadding="8" border="0">
  <colgroup>
    <col span="4">
    <col span="1" bgcolor="lightblue">
    <col span="1" bgcolor="lightgreen">
  </colgroup>
  <tr bgcolor="lightgray">
    <th colspan="4">Características de exemplo</th>
    <th colspan="1">Rótulo</th>
    <th colspan="1">Previsão do modelo</th>
  </tr>
  <tr>
    <td>5.9</td>
<td>3.0</td>
<td>4.3</td>
<td>1.5</td>
<td align="center">1</td>
<td align="center">1</td>
  </tr>
  <tr>
    <td>6.9</td>
<td>3.1</td>
<td>5.4</td>
<td>2.1</td>
<td align="center">2</td>
<td align="center">2</td>
  </tr>
  <tr>
    <td>5.1</td>
<td>3.3</td>
<td>1.7</td>
<td>0.5</td>
<td align="center">0</td>
<td align="center">0</td>
  </tr>
  <tr>
    <td>6.0</td> <td>3.4</td> <td>4.5</td> <td>1.6</td> <td align="center">1</td>
<td align="center" bgcolor="red">2</td>
  </tr>
  <tr>
    <td>5.5</td>
<td>2.5</td>
<td>4.0</td>
<td>1.3</td>
<td align="center">1</td>
<td align="center">1</td>
  </tr>
  <tr><td align="center" colspan="6">     <b>Figura 4.</b> Um classificador de pinguins com 80% de exatidão.<br> </td></tr>
</table>

### Configure o dataset de teste

A avaliação do modelo é parecida com o treinamento dele. A maior diferença é que o exemplo vem de um *[dataset de teste](https://developers.google.com/machine-learning/crash-course/glossary#test_set)* separado, em vez do dataset de treinamento. Para avaliar de maneira justa a eficácia de um modelo, os exemplos usados para isso precisam ser diferentes dos exemplos usados para treinar o modelo.

O dataset de pinguins não tem um dataset de teste separado. Então, na seção anterior "Baixe o dataset", você dividiu o dataset original em datasets de teste e treinamento. Use o dataset `ds_test_batch` para a avaliação.

### Avalie o modelo com o dataset de teste

Ao contrário da etapa de treinamento, o modelo só avalia uma única [época](https://developers.google.com/machine-learning/glossary/#epoch) dos dados de teste. O código a seguir itera cada exemplo do dataset de teste e compara a previsão do modelo com o rótulo real. Essa comparação é usada para medir a exatidão do modelo em todo o dataset de teste.

In [ ]:
test_accuracy = tf.keras.metrics.Accuracy()
ds_test_batch = ds_test.batch(10)

for (x, y) in ds_test_batch:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  logits = model(x, training=False)
  prediction = tf.math.argmax(logits, axis=1, output_type=tf.int64)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Você também pode usar a função do Keras `model.evaluate(ds_test, return_dict=True)` para obter informações de exatidão do dataset de teste. 

Ao inspecionar o último lote, por exemplo, você pode observar que as previsões do modelo geralmente estão corretas.


In [ ]:
tf.stack([y,prediction],axis=1)

## Use o modelo treinado para fazer previsões

Você treinou um modelo e "comprovou" que ele é bom — mas não perfeito — em classificar espécies de pinguins. Agora, vamos usar o modelo treinado para fazer algumas previsões de [*exemplos sem rótulos*](https://developers.google.com/machine-learning/glossary/#unlabeled_example), ou seja, exemplos com características, mas sem rótulos.

Na vida real, os exemplos não rotulados podem vir de diferentes origens, incluindo apps, arquivos CSV e feeds de dados. Para este tutorial, forneça manualmente três exemplos não rotulados para prever seus rótulos. Lembre-se de que os números dos rótulos são mapeados para uma representação de nomes, conforme o seguinte:

- `0`: pinguim-de-adélia
- `1`: pinguim-de-barbicha
- `2`: pinguim-gentoo

In [ ]:
predict_dataset = tf.convert_to_tensor([
    [0.3, 0.8, 0.4, 0.5,],
    [0.4, 0.1, 0.8, 0.5,],
    [0.7, 0.9, 0.8, 0.4]
])

# training=False is needed only if there are layers with different
# behavior during training versus inference (e.g. Dropout).
predictions = model(predict_dataset, training=False)

for i, logits in enumerate(predictions):
  class_idx = tf.math.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))